# 2 Extract speech and laughter from audio files

For speech recognition we try the [SpeechBrain](https://github.com/speechbrain/speechbrain) project and OpenAI's [Whisper](https://github.com/openai/whisper) model.

We also try identifying laughter with [Laughter Detection model](https://github.com/jrgillick/laughter-detection) by jrgillick. 

This code here is based on prototypes developed at Sage IDEMS hackathon in 2023 
https://github.com/chilledgeek/ethical_ai_hackathon_2023


In [1]:
import os
import time
import json
import pandas as pd
import utils

In [2]:
videos_in = os.path.join("..","LookitLaughter.test")
data_out = os.path.join("..", "data", "1_interim")


In [4]:
demo_data = os.path.join("..","data", "demo")
AUDIO_FILE = os.path.join(demo_data, "2UWdXP.joke1.rep2.take1.Peekaboo.mp3")
AUDIO_FILE2 = os.path.join(demo_data, "2UWdXP.joke2.rep1.take1.NomNomNom.mp3")
testset = [AUDIO_FILE, AUDIO_FILE2]

In [ ]:
processedvideos = utils.getProcessedVideos(data_out)
processedvideos.head()

## 2.1 Audio extraction with moviepy

The first step is simple. We extract the audio from each video and save it as `mp3` or `wav`. We will use the `moviepy` library to do this. 
This will be helpful for later analysis and regenerating labeled videos with audio.

Note that `moviepy` is a wrapper around `ffmpeg` and `ffmpeg` needs to be installed separately. 

`conda install ffmpeg moviepy`

In [ ]:
forceaudio = False
#output_ext="mp3"
output_ext="wav"

for index, r in processedvideos.iterrows():
    if forceaudio or pd.isnull(r["Audio.file"]):
        audiopath = utils.convert_video_to_audio_moviepy(videos_in,r["VideoID"], data_out, output_ext=output_ext)
        r["Audio.file"] = audiopath
        r["Audio.when"] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
        #update this row in processedvideos dataframe
        processedvideos.loc[index] = r
    else:
        print("Audio already extracted for video: ", r["VideoID"])
        

utils.saveProcessedVideos(processedvideos, data_out)
processedvideos.head()

## 2.2 Speech-to-text 


### 2.2.1 How not do to it  

We tried [SpeechBrain]((https://github.com/speechbrain/speechbrain), [Speech Recognition](https://github.com/Uberi/speech_recognition) and [Sphinx](https://github.com/cmusphinx/pocketsphinx). None of them worked out of the box. Speechbrain was fiddly and not very accurate (with default settings). Rather than trying to improve it. Let's try the OpenAI Whisper model instead.

See our [version 0.5alpha](https://github.com/InfantLab/babyjokes/releases/tag/v0.5alpha-sage) for the code.

### 2.2.2 Speech-to-text using OpenAI Whisper 

There is a free version of the [OpenAI Whisper](https://github.com/openai/whisper) model. It is multilingual (xx languages) and comes in a range of different sizes (and accuracies). We'll try the `base` model. 

Simple tutorial: https://analyzingalpha.com/openai-whisper-python-tutorial 

In [ ]:
import whisper
model = whisper.load_model("base")

In [ ]:
def whisper_transcribe(audio_file, save_path, saveJSON = True):
    result = model.transcribe(audio_file, verbose = True)
    if saveJSON:
        basename = os.path.basename(audio_file)
        filename, ext = os.path.splitext(basename)
        jsonfile = os.path.join(save_path,filename,".json")
        with open(jsonfile, "w") as f:
            json.dump(result, f)
        return jsonfile, result
    else:
        return result

In [ ]:
processedvideos = utils.getProcessedVideos(data_out)
processedvideos.head()

In [ ]:
for index, r in processedvideos.iterrows():
    if pd.isnull(r["Speech.file"]) and not pd.isnull(r["Audio.file"]):
        speechpath, result = whisper_transcribe(r["Audio.file"],save_path=data_out)
        r["Speech.file"] = speechpath
        r["Speech.when"] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
        #update this row in processedvideos dataframe
        processedvideos.loc[index] = r
        
utils.saveProcessedVideos(processedvideos, data_out)

In [ ]:
processedvideos.head()

## 2.3 Speaker annotation with PyAnnote

We'll use the [PyAnnote](https://github.com/pyannote/pyannote-audio) library to annotate the speakers in the audio. This library listens to the audio and tries to identify the individual speakers. It provides a 'diarization' of the audio. Identifying the start and end of each utterance and assigning them to particular speakers. It can handle multiple speakers and overlapping speech. 

`conda install pyannote.audio -c pyannote`

or 

`pip install pyannote.audio`

To access the pretrained models you need a HuggingFace API token. Store this in .env file. 

In [ ]:
#may want to add two new columns to the processedvideos dataframe
#Diary.file: the path to the diary file
#Diary.when: the time when the diary file was created

if ("Diary.file" not in processedvideos.columns):
    processedvideos["Diary.file"] = None
if ("Diary.when" not in processedvideos.columns):
    processedvideos["Diary.when"] = None    

utils.saveProcessedVideos(processedvideos, data_out)
processedvideos.head()


In [ ]:
forceDiary = True

for index, r in processedvideos.iterrows():
    if (forceDiary or pd.isnull(r["Diary.file"])) and not pd.isnull(r["Audio.file"]):
        diary = diarize_audio(r["Audio.file"])
        #save the diaryjson to a file
        basename = os.path.basename(r["Audio.file"])
        filename, ext = os.path.splitext(basename)
        diarypath = os.path.join(data_out, filename + ".diary.rttm")
        with open(diarypath, "w") as f:
            diary.write_rttm(f)
        r["Diary.file"] = diarypath
        r["Diary.when"] = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime())
        #update this row in processedvideos dataframe
        processedvideos.loc[index] = r

utils.saveProcessedVideos(processedvideos, data_out)
processedvideos.head()

In [ ]:
# we can use load_rttm to reload the rttm file

# Load the diarization output from the RTTM file
_, diary = load_rttm("output.rttm").popitem()

diary

# 2.4 Voice fundamental frequency (F0) estimation

A simvple estimate is easy to get from `librosa`. So let's include it. Will use diarization to do it per utterance.

In [ ]:
def find_f0(audio_file):
    """
    Extract the fundamental frequency (F0) from an audio file.
    Args:
        audio_file (str): The path to the audio file.
    Returns:
        np.array: The fundamental frequency values.
    """
    # Load the audio file
    y, sr = librosa.load(audio_file)
    # Extract the fundamental frequency
    f0, voiced_flag, voiced_probs = librosa.pyin(y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))
    return f0



## 2.5 #TODO - Laughter detection

We would like to process videos to identifying laughter with [Laughter Detection model](https://github.com/jrgillick/laughter-detection) by jrgillick. However, want to find a simple way to call that from remote project rather than incorporating code into our own project.



In [ ]:
import laughter_segmenter


def segment_laughter(wav_filename):
        #results[file_prefix]["laughs"] = segment_laughter(wav_filename)

    return results
